In [1]:
from PIL import Image
import numpy as np

# Create a random image with shape (32, 32, 3)
image_array = np.random.rand(32, 32, 3) * 255
image = Image.fromarray(image_array.astype('uint8'))

# Save the image
image.save("data/sample_image.jpg")
print("Sample image saved as data/sample_image.jpg")


ModuleNotFoundError: No module named 'PIL'

In [9]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the CIFAR-10 dataset
(_, _), (test_images, _) = tf.keras.datasets.cifar10.load_data()

# Get the first test image and normalize it to [0, 255]
image_array = (test_images[0]).astype('uint8')

# Convert to an image and save
image = Image.fromarray(image_array)
image.save("data/sample_image.jpg")
print("Sample image saved as data/sample_image.jpg")

Sample image saved as data/sample_image.jpg


In [13]:
from scripts.preprocess import preprocess_image
# from preprocess import preprocess_image
import json

# Preprocess the sample image
image_path = "data/sample_image.jpg"  # Path to your sample image
preprocessed_data = preprocess_image(image_path)

# Save preprocessed data as JSON
with open("test_data.json", "w") as f:
    json.dump({"instances": preprocessed_data.tolist()}, f)

print("Preprocessed data saved to test_data.json")


Preprocessed data saved to test_data.json


In [16]:
import requests
import numpy as np
from PIL import Image
import json

# Define the endpoint for the model
url = "http://localhost:8501/v1/models/image_classifier:predict"

# Class labels (replace this with your actual class labels)
class_labels = ["class_1", "class_2", "class_3", "class_4", "class_5", "class_6", "class_7", "class_8", "class_9", "class_10"]

# Load and preprocess the image
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Assuming your model expects 224x224 input size
    img = np.array(img) / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img.tolist()  # Convert to list for JSON serialization

# Make a prediction
def get_prediction(image_path):
    data = json.dumps({"instances": preprocess_image(image_path)})
    
    # Send the request to the TensorFlow Serving model
    response = requests.post(url, data=data, headers={"content-type": "application/json"})
    
    # Get the predicted probabilities from the response
    predictions = response.json()["predictions"][0]
    
    # Find the index of the class with the highest probability
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_class_index]
    predicted_probability = predictions[predicted_class_index]
    
    # Return the predicted class and probability
    return predicted_class, predicted_probability

# Example usage
image_path = "data/sample_image.jpg"  # Replace with your image path
predicted_class, predicted_probability = get_prediction(image_path)

print(f"Predicted class: {predicted_class}")
print(f"Prediction probability: {predicted_probability}")


Predicted class: class_1
Prediction probability: 0.737392128


In [1]:
def load_labels(label_file_path):
    with open(label_file_path, "r") as file:
        labels = file.readlines()
    return [label.strip() for label in labels]

# Load your actual class labels
class_labels = load_labels("labels.txt") 

In [2]:
class_labels

['airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck']

In [ ]:
import requests
import numpy as np
from PIL import Image
import json
from scripts.preprocess import preprocess_image

# Define the endpoint for the model
url = "http://localhost:8501/v1/models/image_classifier:predict"

# Load class labels from a file (e.g., 'labels.txt')
def load_labels(label_file_path):
    with open(label_file_path, "r") as file:
        labels = file.readlines()
    return [label.strip() for label in labels]

# Load your actual class labels
class_labels = load_labels("labels.txt")  # Replace with the correct path to your label file

# Load and preprocess the image
# def preprocess_image(image_path):
#     img = Image.open(image_path)
#     img = img.resize((224, 224))  # Assuming your model expects 224x224 input size
#     img = np.array(img) / 255.0  # Normalize the image
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     return img.tolist()  # Convert to list for JSON serialization


# Make a prediction
def get_prediction(image_path):
    image_data =  preprocess_image(image_path)
    
    # Send the request to the TensorFlow Serving model
    # response = requests.post(url, data=data, headers={"content-type": "application/json"})
    response = requests.post(url, json={"instances": image_data.tolist()})
    response.raise_for_status()  # Raise an HTTPError for bad responses
   
    # Get the predicted probabilities from the response
    predictions = response.json()['predictions'][0]
    print(predictions)
    # print(predictions['predictions'][0])
    
    # Find the index of the class with the highest probability
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_class_index]
    predicted_probability = predictions[predicted_class_index]
    
    # Return the predicted class and probability
    return predicted_class, predicted_probability

# Example usage
image_path = "data/sample_image2.jpg"  # Replace with your image path
predicted_class, predicted_probability = get_prediction(image_path)

print(f"Predicted class: {predicted_class}")
print(f"Prediction probability: {predicted_probability}")


[0.00141992338, 0.000366750872, 0.0412123948, 0.204308644, 0.0535725653, 0.380493104, 0.288085639, 0.00339526217, 0.00187686924, 0.0252688359]
5
dog
Predicted class: dog
Prediction probability: 0.380493104


In [ ]:
list(class_labels)

'airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck'